This is check the progress for encoding all 128 configurations

In [1]:
import os
from typing import List, Dict, Optional

OFFLINE_DIR = "/scratch/doluk/Compact-Interference-PRAG/offline"


def extract_args_from_filename_dir(path: str) -> Optional[Dict]:
    """
    Extract experiment arguments from a path ending at:
    .../aug_model=<aug_model>/<filename>

    Only accept lr=..._epoch=..._direct
    Skip warm_up
    """
    # Skip warm_up explicitly
    if "/warm_up/" in path:
        return None

    parts = os.path.normpath(path).split(os.sep)
    args = {}

    for p in parts:
        # rank=2_alpha=32
        if p.startswith("rank="):
            for item in p.split("_"):
                if "=" in item:
                    k, v = item.split("=", 1)
                    args[k] = v

        # lr=0.0003_epoch=4_direct  (ONLY accept _direct)
        elif p.startswith("lr="):
            segs = p.split("_")
            if len(segs) != 3 or segs[2] != "direct":
                return None

            if "=" in segs[0]:
                k, v = segs[0].split("=", 1)
                args[k] = v

            if "=" in segs[1]:
                k, v = segs[1].split("=", 1)
                args[k] = v

            args["cot_name"] = "direct"

        # aug_model=...
        elif p.startswith("aug_model="):
            args["augment_model"] = p.split("=", 1)[1]

    # model_name = directory immediately before rank=
    for i, p in enumerate(parts):
        if p.startswith("rank=") and i > 0:
            args["model_name"] = parts[i - 1]
            break
    else:
        return None

    # dataset = directory immediately after rank=
    for i, p in enumerate(parts):
        if p.startswith("rank=") and i + 1 < len(parts):
            args["dataset"] = parts[i + 1]
            break
    else:
        return None

    # filename is the last directory (total / comparison / bridge / inference / etc.)
    args["filename"] = parts[-1]

    return args


def collect_experiments(root_dir: str) -> List[Dict]:
    """
    Walk OFFLINE_DIR and collect unique experiment folders
    stopping at filename level (parent of data_*)
    """
    experiments = []
    seen = set()

    for root, dirs, _ in os.walk(root_dir):
        # Identify filename directories by presence of data_* children
        if any(d.startswith("data_") for d in dirs):
            if root in seen:
                continue
            seen.add(root)

            args = extract_args_from_filename_dir(root)
            if args is None:
                continue

            experiments.append({
                "path": root,
                "encoded": os.listdir(root).__len__(),
                **args
            })

            # Do not descend into data_* directories
            dirs[:] = [d for d in dirs if not d.startswith("data_")]

    return experiments


if __name__ == "__main__":
    experiments = collect_experiments(OFFLINE_DIR)

    print(f"Found {len(experiments)} experiment configs")

    # sanity check
    for e in experiments[:5]:
        print(e)


Found 212 experiment configs
{'path': '/scratch/doluk/Compact-Interference-PRAG/offline/qwen2.5-1.5b-instruct/rank=4_alpha=32/2wikimultihopqa/lr=0.0003_epoch=4_direct/aug_model=qwen2.5-1.5b-instruct/compositional', 'encoded': 300, 'rank': '4', 'alpha': '32', 'lr': '0.0003', 'epoch': '4', 'cot_name': 'direct', 'augment_model': 'qwen2.5-1.5b-instruct', 'model_name': 'qwen2.5-1.5b-instruct', 'dataset': '2wikimultihopqa', 'filename': 'compositional'}
{'path': '/scratch/doluk/Compact-Interference-PRAG/offline/qwen2.5-1.5b-instruct/rank=4_alpha=32/2wikimultihopqa/lr=0.0003_epoch=4_direct/aug_model=qwen2.5-1.5b-instruct/inference', 'encoded': 300, 'rank': '4', 'alpha': '32', 'lr': '0.0003', 'epoch': '4', 'cot_name': 'direct', 'augment_model': 'qwen2.5-1.5b-instruct', 'model_name': 'qwen2.5-1.5b-instruct', 'dataset': '2wikimultihopqa', 'filename': 'inference'}
{'path': '/scratch/doluk/Compact-Interference-PRAG/offline/qwen2.5-1.5b-instruct/rank=4_alpha=32/2wikimultihopqa/lr=0.0003_epoch=4_dire

In [37]:
experiments[0]

{'path': '/scratch/doluk/Compact-Interference-PRAG/offline/qwen2.5-1.5b-instruct/rank=4_alpha=32/2wikimultihopqa/lr=0.0003_epoch=4_direct/aug_model=qwen2.5-1.5b-instruct/compositional',
 'encoded': 300,
 'rank': '4',
 'alpha': '32',
 'lr': '0.0003',
 'epoch': '4',
 'cot_name': 'direct',
 'augment_model': 'qwen2.5-1.5b-instruct',
 'model_name': 'qwen2.5-1.5b-instruct',
 'dataset': '2wikimultihopqa',
 'filename': 'compositional'}

In [2]:
import os
from typing import List, Dict, Optional
import pandas as pd

# =========================
# Configuration
# =========================

OFFLINE_DIR = "/scratch/doluk/Compact-Interference-PRAG/offline"

MODELS = [
    "qwen2.5-1.5b-instruct",
    "llama3.2-1b-instruct",
    "llama3-8b-instruct",
    "qwen2.5-0.5b-instruct",
]

RANKS = [2, 4, 8, 16]
EPOCHS_LIST = [2, 4]
r_eps_pair = [(r, e) for r in RANKS for e in EPOCHS_LIST]

dataset_filename_dict = {
    "popqa": ["total"],
    "hotpotqa": ["total", "comparison", "bridge"],
    "complexwebquestions": ["total"],
    "2wikimultihopqa": ["total", "bridge_comparison", "comparison", "compositional", "inference"],
}

EXPECTED_SAMPLES = 300

# =========================
# Path parsing
# =========================

def extract_args_from_filename_dir(path: str) -> Optional[Dict]:
    """
    Extract experiment arguments from:
    .../<model>/rank=.../<dataset>/lr=..._epoch=..._direct/aug_model=.../<filename>

    Rules:
    - skip warm_up
    - accept only *_direct
    """
    if "/warm_up/" in path:
        return None

    parts = os.path.normpath(path).split(os.sep)
    args = {}

    for p in parts:
        # rank=2_alpha=32
        if p.startswith("rank="):
            for item in p.split("_"):
                if "=" in item:
                    k, v = item.split("=", 1)
                    args[k] = int(v)

        # lr=0.0003_epoch=4_direct
        elif p.startswith("lr="):
            segs = p.split("_")
            if len(segs) != 3 or segs[2] != "direct":
                return None

            args["learning_rate"] = float(segs[0].split("=", 1)[1])
            args["epoch"] = int(segs[1].split("=", 1)[1])
            args["cot_name"] = "direct"

        # aug_model=...
        elif p.startswith("aug_model="):
            args["augment_model"] = p.split("=", 1)[1]

    # model_name = directory before rank=
    for i, p in enumerate(parts):
        if p.startswith("rank=") and i > 0:
            args["model_name"] = parts[i - 1]
            break
    else:
        return None

    # dataset = directory after rank=
    for i, p in enumerate(parts):
        if p.startswith("rank=") and i + 1 < len(parts):
            args["dataset"] = parts[i + 1]
            break
    else:
        return None

    # filename (total / comparison / bridge / inference / etc.)
    args["filename"] = parts[-1]

    return args

# =========================
# Experiment collection
# =========================

def collect_experiments(root_dir: str) -> List[Dict]:
    experiments = []
    seen = set()

    for root, dirs, _ in os.walk(root_dir):
        data_dirs = [d for d in dirs if d.startswith("data_")]
        if not data_dirs:
            continue

        if root in seen:
            continue
        seen.add(root)

        args = extract_args_from_filename_dir(root)
        if args is None:
            continue

        experiments.append({
            "path": root,
            "encoded": len(data_dirs),
            **args,
        })

        # do not descend into data_*
        dirs[:] = [d for d in dirs if not d.startswith("data_")]

    return experiments

# =========================
# Main
# =========================

if __name__ == "__main__":
    # -------------------------
    # Collect experiments
    # -------------------------
    experiments = collect_experiments(OFFLINE_DIR)
    exp_df = pd.DataFrame(experiments)

    print(f"Found {len(exp_df)} experiment configs")

    # -------------------------
    # Build lookup
    # -------------------------
    # (dataset, filename, rank, epoch, model) -> encoded
    exp_lookup = {
        (row.dataset, row.filename, row.rank, row.epoch, row.model_name): row.encoded
        for row in exp_df.itertuples(index=False)
    }

    # -------------------------
    # Build tracking table
    # -------------------------
    dts_fn = [
        f"{dataset}/{fn}"
        for dataset, fns in dataset_filename_dict.items()
        for fn in fns
    ]

    df = pd.DataFrame({"Dataset": dts_fn})

    for r, eps in r_eps_pair:
        col_name = f"r{r}_e{eps}"
        col_values = []

        for dts in dts_fn:
            dataset, filename = dts.split("/")

            model_status = []
            for model in MODELS:
                key = (dataset, filename, r, eps, model)
                encoded = exp_lookup.get(key, None)

                if encoded is None:
                    model_status.append(0)          # not exist
                elif encoded < EXPECTED_SAMPLES:
                    model_status.append("!")        # incomplete
                else:
                    model_status.append(1)          # done

            col_values.append(model_status)

        df[col_name] = col_values

    # -------------------------
    # Output
    # -------------------------
    print(df.head())
    df.to_csv("encoding_progress.csv", index=False)
    df.to_excel('encoding_progress.xlsx', index=False)


Found 212 experiment configs
                     Dataset         r2_e2         r2_e4         r4_e2  \
0                popqa/total  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]   
1             hotpotqa/total  [0, 0, 0, 0]  [0, 0, 0, 0]  [0, 0, 0, 0]   
2        hotpotqa/comparison  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]   
3            hotpotqa/bridge  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]   
4  complexwebquestions/total  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]   

          r4_e4         r8_e2         r8_e4        r16_e2        r16_e4  
0  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]  
1  [0, 0, 0, 0]  [0, 0, 0, 0]  [0, 0, 0, 0]  [0, 0, 0, 0]  [0, 0, 0, 0]  
2  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]  
3  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, !, 0, 1]  [1, 1, 0, 1]  [!, 1, 0, 1]  
4  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]  [1, 1, 0, 1]  
